In [5]:
%matplotlib notebook
import time
import sklearn
import pandas as pd
import numpy as np
import category_encoders
from numpy.linalg.linalg import LinAlgError
from collections import Counter
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression as lr
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as confu
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.cluster import KMeans
from itertools import combinations as Cb


In [15]:
train_dummy.shape[1]

117

In [16]:
# Normalize data
def normalization(training_numpy, testing_numpy):
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy)
    testing_x_scaled = min_max.transform(testing_numpy)
    #training_norm = pd.DataFrame(x_scaled, columns = columns)
    #testing_norm = pd.DataFrame(testting_x_scaled, columns = columns)
    return (x_scaled, testing_x_scaled)

def pca_function(rate, data):
    
    try:
        pca = PCA()
        pca.fit(data)
    except LinAlgError:
        pca = PCA(n_components=data.shape[1]/2)
        
    for thres_n in xrange(1,len(data)):
        if sum(pca.explained_variance_ratio_[:thres_n])>rate:
            pca_n = thres_n
            break
    
    pca = PCA(n_components=pca_n)
    pca_data = pca.fit_transform(data)
    return (pca_data, pca)

def lda_whole_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


def lda_function(X, y, n_cluster=2):
    
    original_y = y.copy()
    anomaly_index = []
    
    
    for uni_label in np.unique(y):
        anomaly_index.append(np.where(y == uni_label)[0])

    #km_anomaly = KMeans(n_clusters=n_cluster)
    km_anomaly_model = []
    anomaly_label = []
    for i in range(len(anomaly_index)):
        km_anomaly = KMeans(n_clusters=n_cluster)
        km_anomaly_model.append(km_anomaly.fit(X.iloc[anomaly_index[i]], y[anomaly_index[i]]))
        anomaly_label.append(km_anomaly.labels_)
        
    #print len(anomaly_label),np.unique(anomaly_label[0])
    for i in range(len(anomaly_index)):
        km_uni_len = len(np.unique(anomaly_label[i]))
        for (j,now_label) in zip(range(n_cluster*(i+1), n_cluster*(i+1)-n_cluster, -1), range(n_cluster-1,-1,-1)):
            #print "i",i,"j:",j, "now_label:", now_label
            anomaly_label[i][np.where(anomaly_label[i] == now_label)[0]] = j
        y[anomaly_index[i]] = anomaly_label[i]
    #print np.unique(y)
    #print len(np.where(normal_label == 0)[0]),len(np.where(normal_label == 1)[0]), len(np.where(anomaly_label == 2)[0]), len(np.where(anomaly_label == 3)[0])
    #print np.unique(y)
    #return (X,y)
    #print np.unique(y)
    
    #print lda.n_components
    try:
        lda = LDA()
        lda_data = lda.fit_transform(X, y)
    except ValueError:
        lda = LDA()
        print np.unique(original_y)
        lda_data = lda.fit_transform(X, original_y)
    except LinAlgError:
        lda = LDA()
        lda_data = lda.fit_transform(X, original_y)
        
    #print lda_data.shape
    return (lda_data, lda)


'''
def pca_function(n, data):
    pca = PCA(n_components=n)
    pca_data = pca.fit_transform(data)
    return (pca_data.flatten(), pca)

def lda_function(X, y):
    lda = LDA()
    lda_data = lda.fit_transform(X, y)
    return (lda_data.flatten(), lda)
'''
def replace_value(training_normal_numpy, key, loc):  
    training_normal_numpy[loc][key] = 1
    
def del_zero_column(df_train, df_test):
    new_df_test = pd.DataFrame(np.zeros([df_test.shape[0], df_train.shape[1]]),columns=df_train.columns)
    new_df_test[list(set(df_train.columns) & set(df_test.columns))] = df_test[list(set(df_train.columns) & set(df_test.columns))]
    #df_test = df_test[df_train.columns]
    return (df_train, new_df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]
#def del_zero_column(df_train, df_test):
#    df_train = df_train.loc[:, (df_train != 0).any(axis=0)]
#    df_test = df_test[df_train.columns]
#    return (df_train, df_test)

def combine_function(x):
    new_x = ''
    for i in range(len(x)):
        new_x = new_x + str(x[i]) +'_'
    return new_x[:-1]

def all_lda(train_combine, label_num, n_cluster = 2):
    train_all_dummy = pd.get_dummies(train_combine)
    #train_all_dummy = category_encoders.OneHotEncoder(cols=train_combine.columns.tolist()).fit_transform(train_combine)
    (train_lda, lda) = lda_whole_function(train_all_dummy, np.array(label_num), n_cluster=n_cluster)
    clf=BernoulliNB()
    start = time.time()
    
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score , time.time()-start)

def divide_lda(train_combine, label_num, n_cluster = 2):
    train_column= train_combine.columns
    for col in train_combine.columns:
        train_dummy.append(pd.get_dummies(train_combine[col]))
        #train_dummy.append(category_encoders.OneHotEncoder(cols=[col]).fit_transform(train_combine[[col]]))
    train_lda = pd.DataFrame(index=range(len(label_num)))
    for i in range(len(train_combine.columns)):
        #print train_dummy[i].shape
        (now_train_lda, lda) = lda_whole_function(train_dummy[i], np.array(label_num), n_cluster=3)
        now_lda_col_name = []
        for j in range(len(now_train_lda[0])):
            train_lda[train_column[i]+'_'+str(j+1)] = np.array(now_train_lda)[:,j]
    start = time.time()
    score = np.average(cross_val_score(clf, train_lda, label_num, cv=10))
    return (score, time.time()-start)

# DNA

In [3]:
train=pd.read_csv('splice.data.txt', header=None)
train.columns=['label', 'name', 'dna']
key_str = {}
for (i,label_name) in zip(range(len(np.unique(train['label']))), np.unique(train['label'])):
    key_str[label_name] = i
    
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
train_dummy = []
    
train['dna'] = train['dna'].map(lambda x: list(str(x).strip()))
for idx in xrange(60):
    train['dna_%d'% (idx,)] = train['dna'].map(lambda x: x[idx])

train_column = train.columns[3:]
train = train[train_column]

result = pd.DataFrame(columns=['score', 'time'])


#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    
train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for rate in [0.9,0.8,0.7]:
    total_score = []
    total_time = []
    for iteration in range(5):
        start = time.time()
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_pca, pca) = pca_function(rate, train_dummy.iloc[train_val_index])        
            clf.fit(train_pca, val_label[train_val_index])
            test_pca = pca.transform(train_dummy.iloc[test_val_index])
            now_score.append(accuracy_score(clf.predict(test_pca), val_label[test_val_index]))
        exe_time = time.time() - start    
        #(score, exe_time) = all_lda(train_combine, label_num, cluster)       

        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "rate:", rate
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(rate)] = [np.average(total_score), np.average(total_time)]

result.to_csv('dna_pca_rate.csv')

rate: 0.9
score: 0.82269338135
time: 25.9504734039
rate: 0.8
score: 0.8222696862
time: 20.9400804043
rate: 0.7
score: 0.821126159347
time: 13.3753690243


# Mushroom

In [17]:
mush_columns = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size',\
               'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',\
               'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type',\
               'spore-print-color', 'population', 'habitat']
train = pd.read_csv('agaricus-lepiota.data',header=None)
train.columns = mush_columns
key_str = {'e':0, 'p':1}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['score', 'time'])


#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    
train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for rate in [0.9,0.8,0.7]:
    total_score = []
    total_time = []
    for iteration in range(5):
        start = time.time()
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_pca, pca) = pca_function(rate, train_dummy.iloc[train_val_index])        
            clf.fit(train_pca, val_label[train_val_index])
            test_pca = pca.transform(train_dummy.iloc[test_val_index])
            now_score.append(accuracy_score(clf.predict(test_pca), val_label[test_val_index]))
        exe_time = time.time() - start    
        #(score, exe_time) = all_lda(train_combine, label_num, cluster)       

        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "rate:", rate
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(rate)] = [np.average(total_score), np.average(total_time)]

result.to_csv('mush_pca_rate.csv')

rate: 0.9
score: 0.886588286359
time: 5.06185278893
rate: 0.8
score: 0.881102370834
time: 4.21201767921
rate: 0.7
score: 0.878601820895
time: 3.08991117477


In [12]:
train_dummy.shape

(8124, 117)

In [14]:
pca = PCA(n_components=50)
pca.fit(train_dummy.iloc[train_val_index])

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

# Car

In [18]:
car_columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'label']
train = pd.read_csv('car.data',header=None)
train.columns = car_columns
key_str = {'unacc':0, 'acc':1, 'good':2, 'vgood':3}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key == label else None,key_str.keys()), train['label'])
del train['label']

result = pd.DataFrame(columns=['score', 'time'])

#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    
train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for rate in [0.9,0.8,0.7]:
    total_score = []
    total_time = []
    for iteration in range(5):
        start = time.time()
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_pca, pca) = pca_function(rate, train_dummy.iloc[train_val_index])        
            clf.fit(train_pca, val_label[train_val_index])
            test_pca = pca.transform(train_dummy.iloc[test_val_index])
            now_score.append(accuracy_score(clf.predict(test_pca), val_label[test_val_index]))
        exe_time = time.time() - start    
        #(score, exe_time) = all_lda(train_combine, label_num, cluster)       

        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "rate:", rate
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(rate)] = [np.average(total_score), np.average(total_time)]

result.to_csv('car_pca_rate.csv')

rate: 0.9
score: 0.705478892234
time: 0.290058040619
rate: 0.8
score: 0.702971440825
time: 0.297905778885
rate: 0.7
score: 0.703325178414
time: 0.291408014297


# Botnet

In [19]:
column_str = ['Proto', 'State','Dir','Dport']
train = pd.read_csv('botnet_5.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])
train = train[column_str]
train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]

result = pd.DataFrame(columns=['score', 'time'])
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    
train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for rate in [0.9,0.8,0.7]:
    total_score = []
    total_time = []
    for iteration in range(5):
        start = time.time()
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_pca, pca) = pca_function(rate, train_dummy.iloc[train_val_index])        
            clf.fit(train_pca, val_label[train_val_index])
            test_pca = pca.transform(train_dummy.iloc[test_val_index])
            now_score.append(f1_score(clf.predict(test_pca), val_label[test_val_index]))
        exe_time = time.time() - start    
        #(score, exe_time) = all_lda(train_combine, label_num, cluster)       

        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "rate:", rate
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(rate)] = [np.average(total_score), np.average(total_time)]

result.to_csv('bot5_pca_rate.csv')

rate: 0.9
score: 0.673220916713
time: 1.33805937767
rate: 0.8
score: 0.710314093438
time: 1.20119633675


/usr/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


rate: 0.7
score: 0.0
time: 1.29103636742


In [20]:
column_str = ['Proto', 'State','Dir','Dport']
train = pd.read_csv('botnet_6.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])
train = train[column_str]
train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]

result = pd.DataFrame(columns=['score', 'time'])
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    
train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
for rate in [0.9,0.8,0.7]:
    total_score = []
    total_time = []
    for iteration in range(5):
        start = time.time()
        now_score = []
        for train_val_index, test_val_index in skf:    

            (train_pca, pca) = pca_function(rate, train_dummy.iloc[train_val_index])        
            clf.fit(train_pca, val_label[train_val_index])
            test_pca = pca.transform(train_dummy.iloc[test_val_index])
            now_score.append(f1_score(clf.predict(test_pca), val_label[test_val_index]))
        exe_time = time.time() - start    
        #(score, exe_time) = all_lda(train_combine, label_num, cluster)       

        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "rate:", rate
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(rate)] = [np.average(total_score), np.average(total_time)]

result.to_csv('bot6_pca_rate.csv')

rate: 0.9
score: 0.974290850288
time: 3.74724397659
rate: 0.8
score: 0.974295998172
time: 3.49696464539
rate: 0.7
score: 0.784202274417
time: 3.26707596779


In [21]:
column_str = ['Proto', 'State','Dir','Dport']
train = pd.read_csv('botnet_7.binetflow')

key_str = {'Normal':0, 'Botnet':1, 'Background':2}
label_num = []
map(lambda label: map(lambda key: label_num.append(key_str[key]) if key in label else None,key_str.keys()), train['Label'])
train = train[column_str]
train_index = np.array(np.where(np.array(label_num) == 0)[0].tolist() + np.where(np.array(label_num) == 1)[0].tolist())
train = train.iloc[train_index]
train = train.fillna(0)
label_num = np.array(label_num)[train_index]

result = pd.DataFrame(columns=['score', 'time'])
#clf = SVC(kernel='linear', C=1.0)
clf = BernoulliNB()

skf = StratifiedKFold(np.array(label_num), n_folds=10)
val_label = np.array(label_num)
    
train_dummy = category_encoders.OneHotEncoder().fit_transform(train)
print train_dummy.shape
for rate in [0.9,0.8,0.7]:
    total_score = []
    total_time = []
    for iteration in range(5):
        start = time.time()
        now_score = []
        for train_val_index, test_val_index in skf:    
            (train_pca, pca) = pca_function(rate, train_dummy.iloc[train_val_index]) 
            #print train_pca.shape
            clf.fit(train_pca, val_label[train_val_index])
            test_pca = pca.transform(train_dummy.iloc[test_val_index])
            now_score.append(f1_score(clf.predict(test_pca), val_label[test_val_index]))
        exe_time = time.time() - start    
        #(score, exe_time) = all_lda(train_combine, label_num, cluster)       

        total_score.append(np.average(now_score))
        total_time.append(exe_time)
    print "rate:", rate
    print "score:", np.average(total_score)
    print "time:", np.average(total_time)
    result.loc[str(rate)] = [np.average(total_score), np.average(total_time)]

result.to_csv('bot7_pca_rate.csv')

(1740, 44)
rate: 0.9
score: 0.0
time: 0.420140171051
rate: 0.8
score: 0.0
time: 0.419669771194
rate: 0.7
score: 0.0
time: 0.422531414032


In [17]:
np.bincount(val_label)

array([1677,   63])